In [143]:
import pyodbc
from datetime import date, timedelta, datetime
import pandas as pd

pd.set_option('display.max_colwidth', 0)
pd.options.display.max_columns = 22

## Inicialização do Dataframe com os Erros encontrados

Dataframe que armazena todos os erros encontrados neste notebook, no fim da execução este dataFrame estara preenchido com todos os erros encontrados em cada etapa de validação.

Descrição de cada campo do dataFrame:

- **Erro**:  nome ou descrição curta do erro, geralmente no padrão: **[TABELA] Nome do erro**.
- **Descrição**:  descrição mais detalhada sobre o erro encontrado.
- **Informações**:  contém informações sobre como localizar e corrigir o erro especificado, geralmente esse campo é preenchido com dicionários ou objetos contendo múltiplas informações.

In [144]:
erros = pd.DataFrame(columns=['Erro', 'Descricao', 'Informacoes'])

## Extração de dados
Etapa para conexão com o banco de dados e execução das querys para extração dos dados necessários para validações.

In [145]:
data_inicial = pd.Timestamp.today().date()
emissor = ''
conn_str  = 'DRIVER={SQL Server};SERVER=;DATABASE='+emissor+';Integrated Security=true'
print('> Data inicial para análise: ', data_inicial, '\n> Emissor: ', emissor)

> Data inicial para análise:  2020-04-10 
> Emissor:  pagseguro


In [146]:
try:
    conn = pyodbc.connect(conn_str)
    print('>Servidor Conectado')
except Exception as ex:
    print('>Falha na conexão.')
    print("Erro: "+str(ex))

>Servidor Conectado


#### Consultas

In [147]:
try:
    query_controle_processos = """
    SELECT * 
    FROM ControleProcessos 
    WHERE 
    DataMovimento >= '{}'
    ORDER BY
    DataMovimento ASC
    """.format(
        data_inicial
    )

    df_controleprocessos = pd.read_sql_query(query_controle_processos, conn)
    df_ctrl_processos = df_controleprocessos[df_controleprocessos.FlagCorte == 1]

    query_dias_vencimento = """
    SELECT * FROM Vencimentos
    """
    df_dias_vencimento = pd.read_sql_query(query_dias_vencimento, conn)
except Exception as ex:
    print("Erro: " + str(ex))

In [148]:
try:
    query_feriados = """ 
    SELECT *
    FROM Feriados
    WHERE
    Data >= '{}'
    ORDER BY
    Data ASC
    """.format(data_inicial)
    
    df_feriados = pd.read_sql_query(query_feriados, conn)
except Exception as ex:
    print("Erro: "+str(ex))

In [149]:
try:
    query_controle_vencimentos = """ 
    SELECT * 
    FROM ControleVencimentos
    WHERE
    DataVencimento >= '{}'
    ORDER BY
    DataVencimento ASC
    """.format(data_inicial)
    
    df_controlevencimentos = pd.read_sql_query(query_controle_vencimentos, conn)
    df_ctrl_vencimentos = pd.read_sql_query(query_controle_vencimentos, conn)
except Exception as ex:
    print("Erro: "+str(ex))

In [150]:
conn.close()

### Conversão de datas na ControlProcessos e ControleVencimentos

Existem colunas referete a datas que estão em um formato string e com uma formatação diferente da padrão (YY-mm-dd), logo, se faz necessário a conversão dos campos para o formato correto para aplicar as validações.

In [151]:
try:  
    df_controleprocessos['DataMovimento'] = df_controleprocessos['DataMovimento'].apply(lambda date: date.date())
    df_controleprocessos['VencimentoPadraoFaturar'] = pd.to_datetime(df_controleprocessos['VencimentoPadraoFaturar'], format='%d/%m/%Y').apply(lambda date: pd.Timestamp(date).date() if date != None else pd.NaT)
    df_controleprocessos['VctoVencimentoPadrao'] = pd.to_datetime(df_controleprocessos['VctoVencimentoPadrao'], format='%d/%m/%Y').apply(lambda date: pd.Timestamp(date).date() if date != None else pd.NaT)
    df_controleprocessos['VencimentoPadraoFaturar'] = df_controleprocessos['VencimentoPadraoFaturar'].apply(lambda date: pd.Timestamp(date).date() if date != None else pd.NaT)
    df_controleprocessos['VencimentoPadraoCortar'] = df_controleprocessos['VencimentoPadraoCortar'].apply(lambda date: pd.Timestamp(date).date() if date != None else pd.NaT)
    df_controleprocessos['ProximoVencimentoPadrao'] = df_controleprocessos['ProximoVencimentoPadrao'].apply(lambda date: pd.Timestamp(date).date() if date != None else pd.NaT)
    df_controleprocessos['VencimentoPosProximo'] = df_controleprocessos['VencimentoPosProximo'].apply(lambda date: pd.Timestamp(date).date() if date != None else pd.NaT)
   
    df_controlevencimentos['DataPrevistaCorte'] = df_controlevencimentos['DataPrevistaCorte'].apply(lambda date: date.date())
    df_controlevencimentos['DataVencimento'] = df_controlevencimentos['DataVencimento'].apply(lambda date: date.date())
    df_controlevencimentos['DataPrevistaFaturamento'] = df_controlevencimentos['DataPrevistaFaturamento'].apply(lambda date: date.date())
    df_controlevencimentos['DataPrevistaVencimento'] = df_controlevencimentos['DataPrevistaVencimento'].apply(lambda date: date.date())
except Exception as ex:
    print("Erro: "+str(ex))

## Funções de validação

Funções para validação de:
- Dia de semana
- Feriado
- Ausência de processos cadastrados
- Ausência de vencimentos cadastrados
- Ausência de feriados cadastrados
- Validação de Movimento de corte
- Validação de Vencimento
- Registros duplicados
- Próximo dia útil

In [152]:
days = {
    1: 'Segunda',
    2: 'Terça',
    3: 'Quarta',
    4: 'Quinta',
    5: 'Sexta',
    6: 'Sábado',
    7: 'Domingo'
}

def verifica_dia_de_semana(data):
    '''
            Função para verificar se a data passada está em um dia útil.
    
    Output:
    
    {
        e_dia_de_semana = retorna True se a data passada for um dia de semana, False caso contrário
        dia = retorna nome do dia da data passada ('segunda', 'terca', ...)
    }
    '''
    e_dia_de_semana = data.isoweekday() in range(1,6)
    dia = days[data.isoweekday()]
    return {'e_dia_de_semana': e_dia_de_semana, 'dia': dia}

In [153]:
def verifica_feriado(data):
    '''
            Função para verificar se a data passada é um feriado cadastrado na tabela Feriados.
    
    Output:
    
    {
        e_feriado: retorna True caso o parâmetro data seja um feriado
        feriado: retorna ID, Data e Descricao do feriado caso o parâmetro data seja um feriado, None caso contrário
    }
    '''
    data = pd.Timestamp(data)
    feriado = df_feriados.loc[df_feriados['Data'] == data]
    if not feriado.empty:
        found = feriado.iloc[0]
        return {'e_feriado': True, 'feriado': {'Id_Feriado': found['Id_Feriado'], 'Data': found['Data'].strftime('%Y-%m-%d'), 'Descricao': found['Descricao']}}
    else:
        return {'e_feriado': False}


In [154]:
def verifica_tipo_movimento(tipo_mov, data_mov):
    '''
            Função que faz a verificação do tipoMovimento passado com a DataMovimento referente ao processo.
            
    Output:
    
    {
        'tipo_mov_valido': True apenas se o movimento passado estiver correto referente a data_mov passada. 
        'Movimento': Retorna o tipo do movimento passado -> ['FINAL DE SEMANA', 'DIA DE SEMANA', 'FERIADO']
        'Informacoes': 
    }
    
    '''
    
    if tipo_mov == 0: # TipoMovimento == 0 -> Movimento de fim de semana
        mov_fds = verifica_dia_de_semana(data_mov)
        if mov_fds['e_dia_de_semana']:
            return {'tipo_mov_valido': False, 
                    'Movimento': 'FINAL DE SEMANA',  
                    'Informacoes': {
                        'Emissor': emissor,
                        'TipoMovimento': f"[{tipo_mov}] -> Movimento referente a um FINAL DE SEMANA",
                        'DataMovimento': data_mov.strftime('%Y-%m-%d'),
                        'Dia_da_semana': mov_fds['dia']
                    }}
        else:
            return {'tipo_mov_valido': True}
    elif tipo_mov == 1: # TipoMovimento == 1 -> Movimento de dia de semana
        mov_week_day = verifica_dia_de_semana(data_mov)
        if not mov_week_day['e_dia_de_semana']:
            return {'tipo_mov_valido': False, 
                    'Movimento': 'DIA DE SEMANA',
                    'Informacoes': {
                        'Emissor': emissor,
                        'TipoMovimento': f"[{tipo_mov}] -> Movimento referente a um DIA DE SEMANA",
                        'DataMovimento': data_mov.strftime('%Y-%m-%d'),
                        'Dia_da_semana': mov_week_day['dia']
                    }}
        else:
            return {'tipo_mov_valido': True}
    elif tipo_mov == 2: # TipoMovimento == 2 -> Movimento de feriado
        mov_feriado = verifica_feriado(data_mov)
        if not mov_feriado['e_feriado']:
            return {'tipo_mov_valido': False, 
                    'Movimento': 'FERIADO', 
                    'Informacoes': {
                        'Emissor': emissor,
                        'TipoMovimento': f"[{tipo_mov}] -> Movimento referente a um FERIADO",
                        'DataMovimento': data_mov.strftime('%Y-%m-%d'),
                    }}
        else:
            return {'tipo_mov_valido': True}

In [155]:
def verifica_processos_cadastrados():
    '''
            Função que verifica se há processos cadastrados um ano a frente da data atual.
    '''
    data_ultimo_processo = df_ctrl_processos.iloc[-1]['DataMovimento'].date()
    data_atual = pd.Timestamp.today().date()

    if data_ultimo_processo - data_atual < pd.Timedelta(days=365):
        return {'um_ano_a_frente': False, 'anos': {'ultimo_processo': data_ultimo_processo, 'atual': data_atual}}
    else:
        return {'um_ano_a_frente': True, 'anos': {'ultimo_processo': data_ultimo_processo, 'atual': data_atual}}

In [156]:
def verifica_processos_cadastrados():
    '''
            Função que verifica se há processos cadastrados um ano a frente da data atual.
    '''
    data_ultimo_processo = df_ctrl_processos.iloc[-1]['DataMovimento'].date()
    data_atual = pd.Timestamp.today().date()

    if data_ultimo_processo - data_atual < pd.Timedelta(days=365):
        return {'um_ano_a_frente': False, 'anos': {'ultimo_processo': data_ultimo_processo, 'atual': data_atual}}
    else:
        return {'um_ano_a_frente': True, 'anos': {'ultimo_processo': data_ultimo_processo, 'atual': data_atual}}

In [157]:
def verifica_vencimentos_cadastrados():
    '''
            Função que verifica se há vencimentos cadastrados um ano a frente da data atual.
    '''
    ano_ultimo_vencimento = df_ctrl_vencimentos.iloc[-1]['DataVencimento'].date()
    ano_atual = pd.Timestamp.today().date()

    if ano_ultimo_vencimento - ano_atual < pd.Timedelta(days=365):
        return {'um_ano_a_frente': False, 'anos': {'ultimo_vencimento': ano_ultimo_vencimento, 'atual': ano_atual}}
    else:
        return {'um_ano_a_frente': True, 'anos': {'ultimo_vencimento': ano_ultimo_vencimento, 'atual': ano_atual}}

In [158]:
def verifica_feriados_cadastrados():
    '''
            Função que verifica se há feriados cadastrados um ano a frente da data atual.
    '''
    
    ano_ultimo_feriado = df_feriados.iloc[-1]['Data'].date()
    ano_atual = pd.Timestamp.today().date()
    
    if ano_ultimo_feriado - ano_atual < pd.Timedelta(days=365):
        return {'um_ano_a_frente': False, 'anos': {'ultimo_feriado': ano_ultimo_feriado, 'atual': ano_atual}}
    else:
        return {'um_ano_a_frente': True, 'anos': {'ultimo_feriado': ano_ultimo_feriado, 'atual': ano_atual}}

In [159]:
def validaMovimentoCorte():
    
    # DataFrame apenas com DataMovimento de Corte
    controleprocessos = df_controleprocessos[df_controleprocessos['FlagCorte'] == 1].reset_index()
    controleprocessos.drop('index', axis = 1, inplace = True)

    erros_ctrl_processos = []

    for i, dt in controleprocessos.iterrows():
        #Verifica se a data de movimento esta inserida na ControleVencimento 

        #print('DataMovimento: ', dt['DataMovimento'])
        #print('Tipo DataMovimento: ', type(dt['DataMovimento']))

        #print('Comparacao: ', dt['DataMovimento'] == df_controlevencimentos['DataPrevistaCorte'])

        df_aux = df_controlevencimentos[dt['DataMovimento'] == df_controlevencimentos['DataPrevistaCorte']].reset_index()

        #Se não estiver
        if(df_aux.empty):
            erros_ctrl_processos.append({
                'Erro': '[ControleProcessos] DataMovimento corte não encontrada',
                'Descricao': 'A DataMovimento em dia de Corte não foi encontrada na tabela ControleVencimentos',
                'Informacoes': {
                    'Emissor': emissor,
                    'Id_ControleProcesso': dt['Id_ControleProcesso']
                }
            })

        else: 
            #Valida se os demais campos estão corretos
            #Valida campo VencimentoPadraoFaturar


            if (df_aux['DataVencimento'].iloc[0] != dt['VencimentoPadraoFaturar']):
                erros_ctrl_processos.append({
                    'Erro': '[ControleProcessos] Campo VencimentoPadraoFaturar não validado',
                    'Descricao': 'O campo VencimentoPadraoFaturar não foi validado com a DataVencimento de ControleVencimentos',
                    'Informacoes':{
                        'Emissor': emissor,
                        'Id_ControleProcesso': dt['Id_ControleProcesso'],
                        'Id_ControleVencimentos' : df_aux['Id_ControleVencimentos'].iloc[0]
                    }
                })
            #print('aux', df_aux['DataPrevistaFaturamento'].iloc[0])
            #print('Tipo aux', type(df_aux['DataPrevistaFaturamento'].iloc[0]))
            #print('Movimento', dt['DataMovimento'])
            #print('tipo Movimento', type(dt['DataMovimento']))
            #print('comp', df_aux['DataPrevistaFaturamento'].iloc[0] == dt['DataMovimento'])

            #Valida DataPrevistaFaturamento com DataMovimento
            if (df_aux['DataPrevistaFaturamento'].iloc[0] != dt['DataMovimento']):
                erros_ctrl_processos.append({
                    'Erro': '[ControleProcessos] Campo DataMovimento não validado',
                    'Descricao': 'O campo DataMovimento não foi validado com a DataPrevistaFaturamento em ControleVencimentos',
                    'Informacoes':{
                        'Emissor': emissor,
                        'Id_ControleProcesso': dt['Id_ControleProcesso'],
                        'Id_ControleVencimentos' : df_aux['Id_ControleVencimentos'].iloc[0] 
                    }
                })

            if(df_aux['DataPrevistaVencimento'].iloc[0] != dt['VencimentoPadraoFaturar']):
                erros_ctrl_processos.append({
                    'Erro': '[ControleVencimentos] Campo DataPrevistaVencimento não validado',
                    'Descricao': 'O campo DataPrevistaVencimento não foi validado com o VencimentoPadraoFaturar em ControleProcessos',
                    'Informacoes':{
                        'Emissor': emissor,
                        'Id_ControleProcesso': dt['Id_ControleProcesso'],
                        'Id_ControleVencimentos' : df_aux['Id_ControleVencimentos'].iloc[0] 
                    }
                })
    return erros_ctrl_processos

In [160]:
def validaVencimento():
    
    itens = ['VencimentoPadraoFaturar', 'VencimentoPadraoCortar','ProximoVencimentoPadrao', 'ProximoVencimentoReal', 'VencimentoPosProximo']
    erros_ctrl_processos = []

    #DF_ROTINAS apenas os registros com FlagRotinasVencimento = 1
    df_rotinas = df_controleprocessos[df_controleprocessos['FlagRotinasVencimento'] == 1].reset_index()
    df_rotinas.drop('index', axis = 1, inplace = True)

    for i, data in  df_controleprocessos.iterrows():
        #Valida se a flagCleanUp é 1 em dias de Corte
        if(data['FlagCorte'] == 1):
            #Verifica se a FlagCleanUP não esta setada  
            if (data['FlagCleanUp'] == 0):
                #Monta mensagem de ERRO
                erros_ctrl_processos.append({
                        'Erro': '[ControleProcessos] FlagCleanUP não validada',
                        'Descricao': 'A FlagCleanUP deveria ser 1 em dias de Vencimento (FlagRotinasVencimento = 1)',
                        'Informacoes':{
                            'Emissor': emissor,
                            'Id_ControleProcesso': data['Id_ControleProcesso']
                        }
                    })

            #Verifica se ControleProcessos possui todas as colunas de vencimento preenchidas
            for item in itens:
                if (pd.isnull(data[item])): 
                    erros_ctrl_processos.append({
                        'Erro': f'[ControleProcessos] Coluna {item} não cadastrada',
                        'Descricao': f'A coluna {item} não foi cadastrada em ControleProcessos',
                        'Informacoes':{
                            'Emissor': emissor,
                            'Id_ControleProcesso': (data['Id_ControleProcesso']),
                        }
                    })
                    
    #Verifica se DataMovimento contem o VctoVencimentoPadrao quando for dia de vencimento
    nulls = df_controleprocessos.loc[df_controleprocessos.VctoVencimentoPadrao.isna()]
    diasvenc = [x for x in df_dias_vencimento['Vencimento'].values]
    movimentos_incorretos = [x for x in nulls['DataMovimento'].values if x.day in diasvenc ]
    for x in movimentos_incorretos:
        erros_ctrl_processos.append({
            'Erro': f'[ControleProcessos] VctoVencimentoPadrao não cadastrado',
            'Descricao':f'A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento',
            'Informacoes':{
                'Emissor': emissor,
                'DataMovimento': x
            }
        })


    #
                    
    return erros_ctrl_processos

In [161]:
def registrosDuplicados():
    erros_ctrl_processos = []
    #Colunas que serão comparadas
    colunas = ['DataMovimento', 'VencimentoPadraoFaturar']
    #Primeiros duplicados
    aux = df_controleprocessos[df_controleprocessos.duplicated(subset = colunas, keep = 'first') == True].reset_index()
    #Ultimos duplicados
    aux1 = df_controleprocessos[df_controleprocessos.duplicated(subset = colunas, keep = 'last') == True].reset_index()

    if(not aux.empty):
        for i, data in aux.iterrows():
            indice1 = data['Id_ControleProcesso']
            a = aux1[data['DataMovimento'] == aux1['DataMovimento'] ]
            indice2 = a['Id_ControleProcesso']
            erros_ctrl_processos.append({
                        'Erro': '[ControleProcessos] Registro duplicado',
                        'Descricao': 'Mais de um registro com a mesma DataMovimento e VencimentoPadraoFaturar foram encontrados em ControleProcessos',
                        'Informacoes':{
                            'Emissor': emissor,
                            'Id_ControleProcessos': indice1,
                            'Id_ControleVencimentos': indice2
                        }
                    }) 
    return erros_ctrl_processos

In [162]:
def proximoDiaUtil(d):
    proximo_dia = d + timedelta(days= 7-d.weekday() if d.weekday()>3 else 1)
    feriado = verifica_feriado(proximo_dia)
    
    if(feriado['e_feriado']):
        return proximoDiaUtil(proximo_dia)
    
    return proximo_dia

## Validações

In [163]:
erros_ctrl_processos = []

for index, processo in df_ctrl_processos.iterrows():
    prox_venc_real = processo['ProximoVencimentoReal']
    data_movimento = processo['DataMovimento']
    tipo_movimento = processo['TipoMovimento']
    
    ver_feriado = verifica_feriado(prox_venc_real)
    if ver_feriado['e_feriado']:
        erros_ctrl_processos.append({
            'Erro': '[ControleProcessos] ProximoVencimentoReal em Feriado', 
            'Descricao': f"ProximoVencimentoReal no processo[id: {processo['Id_ControleProcesso']}] está em um Feriado", 
            'Informacoes': {
                'Id_ControleProcesso': processo['Id_ControleProcesso'],
                'ProximoVencimentoReal_ControleProcessos': prox_venc_real.strftime('%Y-%m-%d'),
                'Id_Feriado': ver_feriado['feriado']['Id_Feriado'],
                'Data_Feriado': ver_feriado['feriado']['Data'],
                'Descricao_Feriado': ver_feriado['feriado']['Descricao']
            }
        })
    else:
        ver_dia_semana = verifica_dia_de_semana(prox_venc_real)
        if not ver_dia_semana['e_dia_de_semana']:
            erros_ctrl_processos.append({
                'Erro': '[ControleProcessos] ProximoVencimentoReal em dia não útil', 
                'Descricao': f"ProximoVencimentoReal no processo[id: {processo['Id_ControleProcesso']}] está em um dia não útil", 
                'Informacoes': {
                    'Id_ControleProcesso': processo['Id_ControleProcesso'],
                    'ProximoVencimentoReal_ControleProcessos': prox_venc_real.strftime('%Y-%m-%d'),
                    'Dia_da_semana': ver_dia_semana['dia'],
                }
            })
    
    ver_tipo_mov = verifica_tipo_movimento(tipo_mov = tipo_movimento, data_mov = data_movimento)
    if not ver_tipo_mov['tipo_mov_valido']:
        erros_ctrl_processos.append({
                'Erro': '[ControleProcessos] DataMovimento não válida referente ao TipoMovimento',
                'Descricao': f"Processo[id:{processo['Id_ControleProcesso']}] com TipoMovimento={tipo_movimento}, referente a um {ver_tipo_mov['Movimento']}, está com DataMovimento inconsistente ao seu referido TipoMovimento",
                'Informacoes': ver_tipo_mov['Informacoes']
            })


erros = erros.append(erros_ctrl_processos, ignore_index=True)

## Ausência de dados na ControleProcessos
Verifica se há processos cadastrados um ano a frente

In [164]:
ausencia_processos = verifica_processos_cadastrados()
if not ausencia_processos['um_ano_a_frente']:
    erros = erros.append({
            'Erro': '[ControleProcessos] Ausência de processos cadastrados', 
            'Descricao': 'Processos não estão cadastrados 1 ano a frente', 
            'Informacoes': {
                'Ano_ultimo_processo': ausencia_processos['anos']['ultimo_processo'],
                'Ano_atual': ausencia_processos['anos']['atual']
            }}, ignore_index=True)

## Ausência de dados na ControleVencimentos
Verifica se há vencimentos cadastrados um ano a frente

In [165]:
ausencia_vencimentos = verifica_vencimentos_cadastrados()
if not ausencia_vencimentos['um_ano_a_frente']:
    erros = erros.append({
            'Erro': '[ControleVencimentos] Ausência de vencimentos cadastrados', 
            'Descricao': 'Vencimentos não estão cadastrados 1 ano a frente', 
            'Informacoes': {
                'Ano_ultimo_vencimento': ausencia_vencimentos['anos']['ultimo_vencimento'],
                'Ano_atual': ausencia_vencimentos['anos']['atual']
            }}, ignore_index=True)

In [166]:
resultado = validaMovimentoCorte()
erros = erros.append(resultado, ignore_index=True)

In [167]:
resultado = validaVencimento()
erros = erros.append(resultado, ignore_index=True)

In [168]:
resultado = registrosDuplicados()
erros = erros.append(resultado, ignore_index=True)

In [169]:
erros

,Erro,Descricao,Informacoes
0,[ControleProcessos] Ausência de processos cadastrados,Processos não estão cadastrados 1 ano a frente,"{'Ano_ultimo_processo': 2021-03-31, 'Ano_atual': 2020-04-10}"
1,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-04-25}"
2,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-05-10}"
3,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-06-20}"
4,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-07-05}"
5,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-07-25}"
6,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-08-01}"
7,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-08-15}"
8,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-09-05}"
9,[ControleProcessos] VctoVencimentoPadrao não cadastrado,A Coluna VctoVencimentoPadrao não está cadastrada em um dia de vencimento,"{'Emissor': 'pagseguro', 'DataMovimento': 2020-09-20}"
